In [1]:
SearchIssue.objects.all().delete()
# Profile.objects.all().delete()  # Keeping some profiles would be better
Suggestion.objects.all().delete()
Recommendation.objects.all().delete()
Pairing.objects.all().delete()
SocialApp.objects.all().delete()
Session.objects.all().delete()
ContentType.objects.all().delete()

In [8]:
from collections import Counter
nb = Counter(Rating.objects.values_list('work_id', flat=True))
for model in [Anime, Manga]:
    work_ids = list(model.objects.values_list('id', flat=True))
    work_ids.sort(key=lambda work_id: -nb[work_id])
    model.objects.exclude(id__in=work_ids[:20]).delete()

In [9]:
bundle = []
for artist_ids in Work.objects.values_list('anime__director_id', 'anime__composer_id', 'anime__author_id', 'manga__mangaka_id', 'manga__writer_id'):
    bundle.extend(artist_ids)
kept_artist_ids = list(set([x for x in bundle if x is not None]))
Artist.objects.exclude(id__in=kept_artist_ids).delete()

In [10]:
Artist.objects.count()

54

In [11]:
Work.objects.count()

40

In [12]:
Rating.objects.count()

3098

In [2]:
max_user_id = max(User.objects.values_list('id', flat=True))

In [3]:
import random
chosen = User.objects.order_by('?')[:200]
kept_ids = chosen.values_list('id', flat=True)
User.objects.exclude(id__in=kept_ids).delete()

In [4]:
User.objects.count()

200

In [5]:
from django.utils import timezone
new_ids = random.sample(range(max_user_id + 1, max_user_id + 201), 200)
for user, new_id in zip(User.objects.all(), new_ids):
    old_id = user.id
    user.pk = new_id
    user.username = str(new_id)
    user.is_superuser = False
    user.set_password('mangaki')
    user.email = '%d@mangaki.fr' % new_id
    user.date_joined = timezone.now()
    user.last_login = timezone.now()
    user.save()
    old_user = User.objects.get(id=old_id)
    old_user.rating_set.update(user=user)
    # old_user.profile.user = user
    old_user.delete()

In [7]:
User.objects.all()

[<User: 1848>, <User: 1858>, <User: 1847>, <User: 1956>, <User: 1902>, <User: 1892>, <User: 1853>, <User: 1805>, <User: 1955>, <User: 1856>, <User: 1842>, <User: 1897>, <User: 1970>, <User: 2000>, <User: 1963>, <User: 1836>, <User: 1828>, <User: 1833>, <User: 1821>, <User: 1851>, '...(remaining elements truncated)...']

In [15]:
import json
j = json.load(open('../fixtures/seed_data.json'))
Counter(entry['model'] for entry in j)

Counter({'auth.user': 200,
         'mangaki.anime': 20,
         'mangaki.artist': 54,
         'mangaki.deck': 8,
         'mangaki.editor': 17,
         'mangaki.genre': 34,
         'mangaki.manga': 20,
         'mangaki.page': 4,
         'mangaki.rating': 3098,
         'mangaki.studio': 50,
         'mangaki.work': 40,
         'sessions.session': 1,
         'sites.site': 1})